# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['AngleX', 'AngleY', 'LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [2]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

['Stand', 'Left', 'Crouch', 'Sit', 'Belly', 'Knee', 'StandInit', 'Back', 'Right', 'Frog', 'HeadBack']


In [3]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    print len(data[0])
    return data, target

In [4]:
# load all the data
all_data = []
all_target = []

for i in range(0,len(classes)):
    data,target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)
    
    
print 'total number of data', len(all_data)

10
10
10
10
10
10
10
10
10
10
10
total number of data 200


## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [5]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.99)
training_data = permutation[:n_training_data]

In [6]:
print training_data

[  9  41  28 180 138 127 132 101  42 160 184  55 145 199 188  85 114 117
  89  64 197 186  52  18 187  82 164 119 143 137 124 112 104 108 153  90
 198  86  34 150  50 166  98  27 167  70 159 165 110 147  91 120 170 179
  35 136 113  12  48 152 121 194 149   8 172  94  88 107  17 192  31 174
 140  33   1 151  58  49  81 175 156 185 103 144 126  38  79 102  78 182
 109  87  71  51 183  21 139 162 133 154 142  59  56  10 191  26  24  22
  92 163 181  16   3  13  60  61  74  73  46 111  65  45  76  19  72 125
  69 141  77 106 100  54 158  67  57 129  62  47  37 118  30  43  32 130
 171  80  40  11   7 169  95 168   6  97 105  53 122 116  84 173  68  39
 189 157 177 176 178 195 131  75  44  63 135 155 190  36  23 146  99   2
  96  29  66 134 196  14  93 193 161  25  15 115   5 128   0  20   4 123]


In [7]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [8]:
fit_data = []
fit_target = []
for i in range(0,len(training_data)):
    fit_data.append(all_data[training_data[i]])
    fit_target.append(all_target[training_data[i]])
    
clf.fit(fit_data, fit_target) 

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
  gamma=0.001, kernel='rbf', max_iter=-1, probability=False,
  random_state=None, shrinking=True, tol=0.001, verbose=False)

### predicting

In [9]:
clf.predict(all_data[1]), all_target[1]

(array([0]), 0)

In [10]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

## 4. Evaluate on the test data

In [11]:
expected = []
predicted = []
prediction_data = []
for i in range(0,len(all_data)):
    if( not (i in training_data)):
        expected.append(all_target[i])
        prediction_data.append(all_data[i])
predicted = clf.predict(prediction_data)

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          3       1.00      1.00      1.00         1
          6       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00         2


Confusion matrix:
[[1 0]
 [0 1]]


/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2641: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [12]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [13]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[1]), all_target[1]

(array([0]), 0)